# Wstęp
Zadanie 10 jest pierwszą częścią zajęć laboratoryjnych poświęconych sieciom rekurencyjnym i predykcji z wykorzystaniem danych multimodalnych. Efektem prac będzie sieć rekurencyjna do predykcji kursu kryptowaluty Bitcoin (BTC) w oparciu o dane z giełdy oraz o wyniki analizy emocji komunikatów z mediów społecznościowych, do których również należy utworzyć dedykowany model sieci rekurencyjnej. Plan realizacji etapów wygląda następująco:

1.   EmoTweet - model sieci rekurencyjnej do analizy emocji (10 pkt., laboratorium 10)
2. Agregacja informacji emotywnej i przygotowanie MultiBTC - multimodalnego model sieci rekurencyjnej do predykcji kursu BTC (10 pkt., laboratorium 11)
3. Ewaluacja modelu MultiBTC (10 pkt., laboratorium 12)

Łącznie można otrzymać 30 punktów.

# Cel ćwiczenia

Celem pierwszego etapu prac jest zapoznanie się z podstawową siecią rekurencyjną LSTM. Ze względu na fakt, że model ten będzie wykorzystany do analizy emocji tekstu, w ramach teorii do zadania zostanie omówiony podstawowy mechanizm konwersji słów w tekście do postaci wektorów dystrybucyjnych (tzw. word embeddings) na podstawie rozwiązania o nazwie `fastText`. Modele będą budowane na ogólnodostępnym zbiorze `TweetEval`, zawierającym podzbiory ręcznie anotowanych tweetów przy pomocy etykiet odnoszących się do następujących zjawisk: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance). 

# Warunki zaliczenia

Do zaliczenia pierwszego etapu należy utworzyć następujące modele dla min. 2 wybranych zjawisk:

1.   Model bazowy (regresja logistyczna).
2.   Model rekurencyjny oparty o sieć LSTM.

Wytrenowane modele będą wykorzystane w 2 etapie, dlatego proszę je zachować.

# Wektory dystrybucyjne

W przetwarzaniu języka naturalnego, o wektorach dystrybucyjnych (inaczej osadzeniach lub zanurzeniach, ang. word embeddings) mówi się w kontekście reprezentacji słów w tekście, zazwyczaj w postaci wektora liczb rzeczywistych, który koduje znaczenie słowa. Hipoteza dystrybucyjna, u podstawy której leży większość metod reprezentacji, mówi o tym, że słowa, które często współwystępują, mają podobne znaczenie. Wektory dystrybucyjne można uzyskać za pomocą zestawu technik modelowania języka, w których słowa lub frazy są mapowane do wektorów liczb rzeczywistych. Z reguły polega to na matematycznym zanurzeniu z przestrzeni o wielu wymiarach opisujących słowo (konteksty) do ciągłej przestrzeni wektorowej o znacznie mniejszym wymiarze.

Metody generowania tego odwzorowania obejmują sieci neuronowe, redukcję wymiarowości na macierzy współwystępowania słów, modele probabilistyczne lub jawną reprezentację w kontekście, w którym pojawiają się słowa. Wektory dystrybucyjne, używane jako podstawowa reprezentacja wejściowa tekstu, okazały się istotnie poprawiać jakość w wielu zadaniach NLP, takich jak np. rozpoznawanie nazw własnych, określanie części mowy, rozpoznawanie dziedziny tekstu, czy też rozpoznawanie wydźwięku i emocji w tekście. 

# fastText

[fastText](https://fasttext.cc/) jest biblioteką do efektywnego uczenia modeli reprezentacji wektorowych słów oraz do budowania klasyfikatorów tekstu. Modele językowe można budować z wykorzystaniem dwóch popularnych technik: [Continuous Bag of Words](https://www.kdnuggets.com/2018/04/implementing-deep-learning-methods-feature-engineering-text-data-cbow.html) oraz [Skip-Gram](https://towardsdatascience.com/skip-gram-nlp-context-words-prediction-algorithm-5bbf34f84e0c). 

## Instalacja

Pobranie repozytorium projektu:


In [1]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3938, done.
remote: Counting objects: 100% (997/997), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 3938 (delta 902), reused 855 (delta 850), pack-reused 2941
Receiving objects: 100% (3938/3938), 8.25 MiB | 28.24 MiB/s, done.
Resolving deltas: 100% (2509/2509), done.


Instalacja biblioteki:

In [2]:
!cd fastText && mkdir build && cd build && cmake ..  && make && make install

CMake Deprecation Warning at CMakeLists.txt:9 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /content/fastText/build
[  2%] Building CXX object CMakeFiles/fasttext-shared.dir/src/args.cc.o
[  4%] Building

Instalacja API do Pythona:

In [3]:
!cd fastText && pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/fastText
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4384288 sha256=580967f9850467401767d6e004b1e63e332ac40d1b1e3d5ad9baf86da0e5d002
  Stored in directory: /tmp/pip-ephem-wheel-cache-foiwu33k/wheels/8b/05/af/3cfae069d904597d44b309c956601b611bdf8967bcbe968903
Successfully built fasttext


# Dane do etapu nr 1

## Korpus 
Korpus (zbiór dokumentów) do realizacji etapu nr 1 pochodzi z repozytorium [TweetEval](https://github.com/cardiffnlp/tweeteval). Repozytorium zawiera 7 różnorodnych zbiorów danych, zawierających zanonimizowane wpisy z [Twittera](https://twitter.com), anotowane następującymi zjawiskami: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance). 

In [4]:
from google.colab import drive
import os

drive.mount("/content/drive/")

Mounted at /content/drive/


In [5]:
!unzip drive/MyDrive/GSN_dane/tweeteval.zip -d tweeteval

Archive:  drive/MyDrive/GSN_dane/tweeteval.zip
   creating: tweeteval/tweeteval/tweeteval/
   creating: tweeteval/tweeteval/tweeteval/.git/
   creating: tweeteval/tweeteval/tweeteval/.git/branches/
  inflating: tweeteval/tweeteval/tweeteval/.git/config  
  inflating: tweeteval/tweeteval/tweeteval/.git/description  
 extracting: tweeteval/tweeteval/tweeteval/.git/HEAD  
   creating: tweeteval/tweeteval/tweeteval/.git/hooks/
  inflating: tweeteval/tweeteval/tweeteval/.git/hooks/applypatch-msg.sample  
  inflating: tweeteval/tweeteval/tweeteval/.git/hooks/commit-msg.sample  
  inflating: tweeteval/tweeteval/tweeteval/.git/hooks/fsmonitor-watchman.sample  
  inflating: tweeteval/tweeteval/tweeteval/.git/hooks/post-update.sample  
  inflating: tweeteval/tweeteval/tweeteval/.git/hooks/pre-applypatch.sample  
  inflating: tweeteval/tweeteval/tweeteval/.git/hooks/pre-commit.sample  
  inflating: tweeteval/tweeteval/tweeteval/.git/hooks/pre-merge-commit.sample  
  inflating: tweeteval/tweeteval

## Zawartość korpusu

W katalogu głównym (tweeteval) znajdują się następujące elementy:
*   `datasets` - katalog ze zbiorami danych
   * `emotion` - tweety anotowane emocjami 
     * `mapping.txt` - identyfikatory etykiet oraz ich opis
     * `train_text.txt` - wpisy z Twittera (część ucząca)
     * `train_labels.txt` - etykiety wpisów z Twittera (część ucząca)
     * `test_*.txt, valid_*.txt` - j.w. (część testowa i walidacyjna)
   * `emoji` - tweety anotowane emotikonami
   * `...` - katalogi zawierające tweety anotowane pozostałymi zjawiskami
*   `predictions` - katalog z przykładowymi predykcjami
   * `emotion.txt` - etykiety modelu predykcyjnego dla części testowej danych `emotion`
   * `emoji.txt` - j.w. dla cz. testowej danych `emoji`
   * `...` - j.w. dla pozostałych danych
*   `evaluation_script.py` - skrypt do ewaluacji 

## Model języka

Na potrzeby zadania został przygotowany model Skip-Gram reprezentacji wektorowej słów, zbudowany na wielkim korpusie tweetów dotyczących kursu BTC. Wersja binarna tego modelu dostępna jest w 2 wariantach:
* wektory 100-elementowe (1.7GB, fasttext_tweetmodel_btc_sg_100_en.bin)
* wektory 20-elementowe (350MB, fasttext_tweetmodel_btc_sg_20_en.bin)

Na potrzeby prezentacji przykładowego rozwiązania zostanie wykorzystany mniejszy model. Do realizacji ostatecznego rozwiązania należy wykorzystać większy model. 



# Model bazowy rozpoznawania emocji

Model bazowy, zbudowany z wykorzystaniem narzędzia fastText (oparty o regresję logistyczną), będzie punktem wyjścia do porównania się z modelami opartymi o sieci LSTM, których skonstruowanie i ewaluacja na wybranych zadaniach będzie celem etapu nr 1. 

Pobranie mniejszego modelu reprezentacji języka tweetów:


In [ ]:
# należy wgrać plik z katalogu "dane" o nazwie fasttext_tweetmodel_btc_sg_20_en.bin

Wydobycie słownika wektorów z binarnego modelu języka:

In [6]:
!python fastText/python/doc/examples/bin_to_vec.py drive/MyDrive/GSN_dane/fasttext_tweetmodel_btc_sg_20_en.bin > fasttext_tweetmodel_btc_sg_20_en.vec

Dodanie prefiksu `__label__` do etykiet zbioru `emotion`:

In [28]:
!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/emotion/train_labels.txt > train_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/emotion/test_labels.txt > test_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/emotion/val_labels.txt > val_labels_emo.txt

Przygotowanie zbioru uczącego, testowego i walidacyjnego w formacie `fastText`:

In [30]:
!paste -d " " tweeteval/tweeteval/tweeteval/datasets/emotion/train_text.txt train_labels_emo.txt > train_emo.txt
!paste -d " " tweeteval/tweeteval/tweeteval/datasets/emotion/test_text.txt test_labels_emo.txt > test_emo.txt
!paste -d " " tweeteval/tweeteval/tweeteval/datasets/emotion/val_text.txt val_labels_emo.txt > val_emo.txt

Trenowanie modelu z wykorzystaniem wejścia `train_emo.txt`, z określeniem wyjściowej nazwy modelu `emo_model`, dla wektorów słów o wymiarze `20`, z wykorzystaniem pretrenowanych wektorów z pliku `fasttext_tweetmodel_btc_sg_20_en.vec` i z uruchomieniem dostrajania hiperparametrów na zbiorze walidacyjnym `val_emo.txt`:

In [31]:
!fasttext supervised -input train_emo.txt -output emo_model -dim 20 -pretrainedVectors fasttext_tweetmodel_btc_sg_20_en.vec -autotune-validation val_emo.txt 

Progress: 100.0% Trials:   14 Best score:  0.695187 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  12887
Number of labels: 4
Progress: 100.0% words/sec/thread:   73382 lr:  0.000000 avg.loss:  0.309875 ETA:   0h 0m 0s


Podstawowa ewaluacja modelu z wykorzystaniem `fastText`, wynikiem jest precyzja (P - precision) i kompletność (R - recall) w wariancie [weighted](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html).

In [32]:
!fasttext test emo_model.bin test_emo.txt

N	1421
P@1	0.678
R@1	0.678


Rozszerzona ewaluacja modelu z wykorzystaniem `fastText`, wynikiem jest precyzja (P - precision), kompletność (R - recall) oraz F1-score dla każdej etykiety w wariancie [weighted](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html).

In [33]:
!fasttext test-label emo_model.bin test_emo.txt

F1-Score : 0.752492  Precision : 0.701238  Recall : 0.811828   __label__0
F1-Score : 0.658824  Precision : 0.657963  Recall : 0.659686   __label__3
F1-Score : 0.660661  Precision : 0.714286  Recall : 0.614525   __label__1
F1-Score : 0.367150  Precision : 0.452381  Recall : 0.308943   __label__2
N	1421
P@1	0.678
R@1	0.678


Przygotowanie danych do ewaluacji z wykorzystaniem skryptu dołączonego do zbioru TweetEval:

In [34]:
!mkdir predictions2

In [36]:
!fasttext predict emo_model.bin tweeteval/tweeteval/tweeteval/datasets/emotion/test_text.txt | sed 's/__label__//g' > predictions2/emotion.txt

Uruchomienie ewaluacji. Oprócz wyników P, R, F1 [weighted]((https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)) dla każdej etykiety, otrzymujemy również wyniki w wariancie [macro]((https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)). **Ostateczną miarą (TweetEval Score) jest miara F1-score w wariancie macro i tę miarę proszę traktować jako kluczową przy porównywaniu rozwiązań.**

In [39]:
!python tweeteval/tweeteval/tweeteval/evaluation_script.py --tweeteval_path tweeteval/tweeteval/tweeteval/datasets --predictions_path predictions2 --task emotion

0 {'precision': 0.7012383900928792, 'recall': 0.8118279569892473, 'f1-score': 0.7524916943521596, 'support': 558}
1 {'precision': 0.7142857142857143, 'recall': 0.6145251396648045, 'f1-score': 0.6606606606606606, 'support': 358}
2 {'precision': 0.4523809523809524, 'recall': 0.3089430894308943, 'f1-score': 0.36714975845410625, 'support': 123}
3 {'precision': 0.6579634464751958, 'recall': 0.6596858638743456, 'f1-score': 0.6588235294117647, 'support': 382}
accuracy 0.6776917663617171
macro avg {'precision': 0.6314671258086855, 'recall': 0.5987455124898229, 'f1-score': 0.6097814107196727, 'support': 1421}
weighted avg {'precision': 0.6713513026618537, 'recall': 0.6776917663617171, 'f1-score': 0.67082117557366, 'support': 1421}
------------------------------
TweetEval Score (emotion): 0.6097814107196727


# Budowa modeli EmoTweet

W tej sekcji Państwa zadaniem będzie przygotowanie modeli sieci LSTM oraz modeli bazowych opartych o regresję logistyczną (fastText) dla wybranych 2 zjawisk ze zbioru TweetEval. Dla sieci LSTM kolejne jednostki sieci rekurencyjnej na wejściu dostają reprezentację wektorową kolejnych wyrazów w tekście. Wyjście z ostatniej jednostki podlega klasyfikacji. W celu usprawnienia zadania, przedstawiona zostanie metoda reprezentacji wektorowej tekstu z wykorzystaniem Pythonowego API do narzędzia fastText. Do ewaluacji modeli należy wykorzystać uprzednio zaprezentowany skrypt `tweeteval/evaluation_script.py`.

## Wektoryzacja tekstu


In [2]:
# inicjalizacja biblioteki
import fasttext

In [11]:
# ładowanie modelu
MODEL_PATH = "drive/MyDrive/GSN_dane/fasttext_tweetmodel_btc_sg_20_en.bin"
model = fasttext.load_model(MODEL_PATH)

In [3]:
# wczytanie danych treningowych
import pandas as pd
TRAIN_PATH = 'tweeteval/tweeteval/tweeteval/datasets/emotion/train_text.txt'
train_texts = pd.read_csv(TRAIN_PATH, sep='\t', header=None)
train_texts

,0
0,“Worry is a down payment on a problem you may ...
1,My roommate: it's okay that we can't spell bec...
2,No but that's so cute. Atsu was probably shy a...
3,Rooneys fucking untouchable isn't he? Been fuc...
4,it's pretty depressing when u hit pan on ur fa...
...,...
3252,I get discouraged because I try for 5 fucking ...
3253,The @user are in contention and hosting @user ...
3254,@user @user @user @user @user as a fellow UP g...
3255,You have a #problem? Yes! Can you do #somethin...


In [45]:
# wektoryzacja pierwszego tekstu
first_text = train_texts[0][0]
for word in fasttext.tokenize(first_text):
  print(word, model.get_word_id(word), model.get_word_vector(word))

“Worry -1 [-0.04189867  0.15429688  0.96717507  1.3809655   0.49123076 -0.5447607
 -0.11276884  0.20356484 -1.0640966  -1.6616327   0.03930127 -0.7224096
  0.21334486 -0.5872285   0.2898182   0.81751084 -1.6077403   1.8038087
  0.4850348   1.0643197 ]
is 6 [ 0.24099417  0.13544752  0.7251924   0.32544732  0.27421224  0.31903243
  0.7501186   0.22853182 -0.91543657  0.08587569  0.13866538 -0.38624704
 -0.30637258  0.13666666 -0.43992838 -0.12443608 -1.0383893  -0.06567164
  0.17007533 -0.16708991]
a 7 [-0.00810981 -0.03934941  0.81658655  0.56301105  0.43812367  0.29547286
  0.4691784   0.07483605 -0.58705056  0.28240088 -0.6339584  -0.16187707
 -0.23376046 -0.1245347   0.03071329 -0.07603034 -0.9066614  -0.07007706
  0.4522892  -0.15033531]
down 174 [ 0.9175071  -1.0815151   0.07119758  0.34226617  0.9607946   0.5973182
  0.91058624 -0.32068744 -0.72137564  1.2241784  -0.1882128  -0.23591968
 -0.02596712 -0.10194965 -0.09553405  0.36303622  0.22354192  0.4901933
  0.5405883   0.5965071

Proszę zwrócić uwagę, że fastText jest w stanie przyporządkować reprezentację wektorową nawet dla takich słów, których model języka nie widział w trakcie uczenia (pierwszy token wejściowego tekstu). 

## Model klasyfikacji tekstu LSTM (2 pkt.)

In [3]:
MODEL_PATH = "drive/MyDrive/GSN_dane/fasttext_tweetmodel_btc_sg_100_en.bin"
fast_text_model = fasttext.load_model(MODEL_PATH)

In [4]:
!python fastText/python/doc/examples/bin_to_vec.py drive/MyDrive/GSN_dane/fasttext_tweetmodel_btc_sg_100_en.bin > fasttext_tweetmodel_btc_sg_100_en.vec

In [5]:
!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/emotion/train_labels.txt > train_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/emotion/test_labels.txt > test_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/emotion/val_labels.txt > val_labels_emo.txt

!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/hate/train_labels.txt > train_labels_hate.txt
!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/hate/test_labels.txt > test_labels_hate.txt
!sed 's/^/__label__/g' tweeteval/tweeteval/tweeteval/datasets/hate/val_labels.txt > val_labels_hate.txt

In [6]:
!paste -d " " tweeteval/tweeteval/tweeteval/datasets/emotion/train_text.txt train_labels_emo.txt > train_emo.txt
!paste -d " " tweeteval/tweeteval/tweeteval/datasets/emotion/test_text.txt test_labels_emo.txt > test_emo.txt
!paste -d " " tweeteval/tweeteval/tweeteval/datasets/emotion/val_text.txt val_labels_emo.txt > val_emo.txt

!paste -d " " tweeteval/tweeteval/tweeteval/datasets/hate/train_text.txt train_labels_hate.txt > train_hate.txt
!paste -d " " tweeteval/tweeteval/tweeteval/datasets/hate/test_text.txt test_labels_hate.txt > test_hate.txt
!paste -d " " tweeteval/tweeteval/tweeteval/datasets/hate/val_text.txt val_labels_hate.txt > val_hate.txt

## Trenowanie modeli LSTM dla ZJAWISKO_1 i ZJAWISKO_2 (2 pkt.)
Należy wybrać 2 z 7 dostępnych podzbiorów z [TweetEval](https://github.com/cardiffnlp/tweeteval) anotowanych następującymi zjawiskami: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance).

In [7]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
from tqdm.notebook import tqdm
import numpy as np
import torch
from torch.autograd import Variable
from typing import Tuple

In [8]:
def tokenize(text):
    words = fasttext.tokenize(text)
    return np.asarray([fast_text_model.get_word_vector(x) for x in words])

test_word = fast_text_model.get_word_vector('test')
token_length = len(test_word)

def get_values(df, column):
    lengths = torch.from_numpy(df[column].str.len().to_numpy())
    seq_tensors = Variable(torch.zeros((len(lengths), lengths.max(), token_length))).float()

    vectorized_seqs = df.apply(lambda x: tokenize(x[column]), axis=1).to_numpy()

    for i in range(len(vectorized_seqs)):
        seq = vectorized_seqs[i]
        seq_tensors[i, :len(seq)] = torch.from_numpy(seq)

    seq_lengths, perm_idx = lengths.sort(0, descending=True)
    seq_tensors = seq_tensors[perm_idx]
    
    return seq_tensors, seq_lengths

In [9]:
import pandas as pd

emotion_train_path = 'tweeteval/tweeteval/tweeteval/datasets/emotion/train_text.txt'
X_emotion = pd.read_csv(emotion_train_path, sep='\t', header=None)
X_emotion, lengths_emotion = get_values(X_emotion, 0)

emotion_labels_path = 'tweeteval/tweeteval/tweeteval/datasets/emotion/train_labels.txt'
y_emotion = pd.read_csv(emotion_labels_path, sep='\t', header=None)
y_emotion = torch.from_numpy(y_emotion[0].to_numpy())

hate_train_path = 'tweeteval/tweeteval/tweeteval/datasets/hate/train_text.txt'
X_hate = pd.read_csv(hate_train_path, sep='\t', header=None)
X_hate, lengths_hate = get_values(X_hate, 0)

hate_labels_path = 'tweeteval/tweeteval/tweeteval/datasets/hate/train_labels.txt'
y_hate = pd.read_csv(hate_labels_path, sep='\t', header=None)
y_hate = torch.from_numpy(y_hate[0].to_numpy())

In [10]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.ReLu = nn.ReLU()

    def forward(self, x):
        output, (hidden_state, _) = self.lstm(x)
        output = self.fc(hidden_state[-1])
        output = self.ReLu(output)
        return output

    def train_step(self, x, lengths):
        packed_input = pack_padded_sequence(x, lengths, batch_first=True)
        output = self.forward(packed_input)
        return output



def fit(X, y, lengths,model, optimizer: torch.optim.Optimizer, criterion, epochs: int
):
    for epoch in tqdm(range(epochs)):
        model.train()
        optimizer.zero_grad()
        outputs = model.train_step(X, lengths)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()


        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

In [17]:
model_emotion = LSTMModel(len(test_word), 20, y_emotion.max() + 1)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_emotion.parameters(), lr=0.01)
fit(X_emotion, y_emotion, lengths_emotion, model_emotion, optimizer, loss, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10, Loss: 1.4075205326080322
Epoch 2/10, Loss: 1.3847535848617554
Epoch 3/10, Loss: 1.3650060892105103
Epoch 4/10, Loss: 1.346288800239563
Epoch 5/10, Loss: 1.3276602029800415
Epoch 6/10, Loss: 1.3161383867263794
Epoch 7/10, Loss: 1.305797815322876
Epoch 8/10, Loss: 1.2970695495605469
Epoch 9/10, Loss: 1.294065237045288
Epoch 10/10, Loss: 1.2906346321105957


In [11]:
model_hate = LSTMModel(token_length, 20, y_hate.max() + 1)
y_hate = y_hate[:8993]
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_hate.parameters(), lr=0.01)
fit(X_hate, y_hate, lengths_hate, model_hate, optimizer, loss, 5)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.6816843748092651
Epoch 2/5, Loss: 0.6804516315460205
Epoch 3/5, Loss: 0.6807477474212646
Epoch 4/5, Loss: 0.6808732151985168
Epoch 5/5, Loss: 0.6805976629257202


## Trenowanie modeli LR (fastText) dla ZJAWISKO_1 i ZJAWISKO_2 (2 pkt.)

In [17]:
!fasttext supervised -input train_emo.txt -output emo_model -dim 100 -pretrainedVectors fasttext_tweetmodel_btc_sg_100_en.vec -autotune-validation val_emo.txt

Progress: 100.0% Trials:    5 Best score:  0.697861 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  12887
Number of labels: 4
Progress: 100.0% words/sec/thread:   23332 lr:  0.000000 avg.loss:  0.447785 ETA:   0h 0m 0s


In [22]:
!fasttext supervised -input train_hate.txt -output hate_model -dim 100 -pretrainedVectors fasttext_tweetmodel_btc_sg_100_en.vec -autotune-validation val_hate.txt

Progress: 100.0% Trials:    5 Best score:  0.720000 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  32730
Number of labels: 2
Progress: 100.0% words/sec/thread:   18544 lr:  0.000000 avg.loss:  0.408331 ETA:   0h 0m 0s


## Ewaluacja modeli na danych testowych dla zjawiska ZJAWISKO_1 (2 pkt.)

In [23]:
!mkdir predictions_emo

In [28]:
!fasttext predict emo_model.bin tweeteval/tweeteval/tweeteval/datasets/emotion/test_text.txt | sed 's/__label__//g' > predictions_emo/emotion.txt

In [29]:
!python tweeteval/tweeteval/tweeteval/evaluation_script.py --tweeteval_path tweeteval/tweeteval/tweeteval/datasets --predictions_path predictions_emo --task emotion

0 {'precision': 0.7099236641221374, 'recall': 0.8333333333333334, 'f1-score': 0.766694146743611, 'support': 558}
1 {'precision': 0.7284768211920529, 'recall': 0.6145251396648045, 'f1-score': 0.6666666666666667, 'support': 358}
2 {'precision': 0.4939759036144578, 'recall': 0.3333333333333333, 'f1-score': 0.39805825242718446, 'support': 123}
3 {'precision': 0.6745406824146981, 'recall': 0.6727748691099477, 'f1-score': 0.673656618610747, 'support': 382}
accuracy 0.6917663617171006
macro avg {'precision': 0.6517292678358365, 'recall': 0.6134916688603547, 'f1-score': 0.6262689211120523, 'support': 1421}
weighted avg {'precision': 0.6863938658648139, 'recall': 0.6917663617171006, 'f1-score': 0.6845742392029913, 'support': 1421}
------------------------------
TweetEval Score (emotion): 0.6262689211120523


In [18]:
emotion_test_path = 'tweeteval/tweeteval/tweeteval/datasets/emotion/test_text.txt'
X_emotion = pd.read_csv(emotion_test_path, sep='\t', header=None)
X_emotion, lengths_emotion = get_values(X_emotion, 0)

emotion_labels_path = 'tweeteval/tweeteval/tweeteval/datasets/emotion/test_labels.txt'
y_emotion = pd.read_csv(emotion_labels_path, sep='\t', header=None)
y_emotion = torch.from_numpy(y_emotion[0].to_numpy())

In [12]:
from sklearn.metrics import f1_score
def evaluate(X, y, lengths, model):
    model.eval()
    with torch.no_grad():
        outputs = model.train_step(X, lengths)
        _, predicted_labels = torch.max(outputs, 1)

        accuracy = (predicted_labels == y).sum().item() / len(y) * 100

        f1 = f1_score(y, predicted_labels, average='macro')

        print(f"Accuracy: {accuracy:.2f}%")
        print(f"F1 Score: {f1:.4f}")

In [22]:
evaluate(X_emotion, y_emotion, lengths_emotion, model_emotion)

Accuracy: 39.27%
F1 Score: 0.1410


## Ewaluacja modeli na danych testowych dla zjawiska ZJAWISKO_2 (2 pkt.)

In [63]:
!mkdir predictions_hate

In [64]:
!fasttext predict emo_model.bin tweeteval/tweeteval/tweeteval/datasets/hate/test_text.txt | sed 's/__label__//g' > predictions_hate/hate.txt

In [65]:
!python tweeteval/tweeteval/tweeteval/evaluation_script.py --tweeteval_path tweeteval/tweeteval/tweeteval/datasets --predictions_path predictions_hate --task hate

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
0 {'precision': 0.5676784249384742, 'recall': 0.8055878928987195, 'f1-score': 0.66602502

In [ ]:
hate_test_path = 'tweeteval/tweeteval/tweeteval/datasets/hate/test_text.txt'
X_hate = pd.read_csv(hate_test_path, sep='\t', header=None)
X_hate, lengths_hate = get_values(X_hate, 0)

hate_labels_path = 'tweeteval/tweeteval/tweeteval/datasets/hate/test_labels.txt'
y_hate = pd.read_csv(hate_labels_path, sep='\t', header=None)
y_hate = torch.from_numpy(y_hate[0].to_numpy())

In [ ]:
y_hate = y_hate[:2932]

In [13]:
evaluate(X_hate, y_hate, lengths_hate, model_hate)

Accuracy: 57.97%
F1 Score: 0.3670
